In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [4]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=5, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [5]:
item_subset

[31, 2259, 1203, 5073, 102]

### `/detect_conflict`

In [71]:
import requests
import json

url = 'http://localhost:5000/detect_conflict/'
with open('constraints_1.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [72]:
r.json()['conflict']

'No conflict'

### `/train`

In [68]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

# headers = {'content-type': 'application/json',
#            'Accept-Charset': 'UTF-8'
#           }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

KeyboardInterrupt: 

In [ ]:
data_subset.info()

In [66]:
r.json()['result']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [54]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

,avg_sales,item_id,mae_score,mpe_score,r2_score,rmse_score
0,45.216946,31.0,15.211045,0.336401,0.514597,23.211137
1,176.605667,102.0,44.631816,0.252720,0.562941,61.375934
2,198.348404,1203.0,40.303816,0.203197,0.635932,59.186954
3,97.153709,2259.0,27.536322,0.283430,0.543398,35.557477
4,115.366066,5073.0,28.738143,0.249104,0.480752,37.210351


### `/optimize`

In [164]:
import requests
import json

url = 'http://localhost:5000/optimize/'
with open('constraints_1.json') as f:
    constraints = json.load(f)
    
payload = {'project_id': 1,
           'constraints': constraints,
           'population': 10,
           'max_epoch': 5
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [165]:
best_prices = r.json()['result']
best_prices

[{'1': 4.9,
  '1002': 2.7,
  '1003': 3.3,
  '1004': 3.9,
  '101': 5.3,
  '102': 9.85,
  '11': 5.1,
  '1152': 4.25,
  '1153': 4.95,
  '1195': 3.55,
  '12': 9.7,
  '1203': 3.35,
  '1204': 3.9,
  '1205': 4.2,
  '1221': 3.3,
  '1225': 2.75,
  '1230': 3.6,
  '1231': 4.45,
  '1232': 5.2,
  '1502': 3.95,
  '1503': 4.35,
  '1504': 4.95,
  '16000': 7.45,
  '16001': 11.25,
  '16002': 11.95,
  '16003': 12.65,
  '16016': 8.95,
  '16017': 12.75,
  '16018': 13.45,
  '16019': 14.15,
  '1615': 3.25,
  '196': 5.9,
  '197': 10.6,
  '2': 9.6,
  '200': 6.0,
  '201': 10.6,
  '202': 2.15,
  '203': 5.35,
  '204': 10.25,
  '21': 4.45,
  '22': 9.35,
  '2256': 8.75,
  '2257': 11.95,
  '2258': 12.65,
  '2259': 13.35,
  '2313': 9.35,
  '2315': 13.0,
  '2316': 13.7,
  '2501': 2.8,
  '2511': 3.65,
  '2521': 5.25,
  '2531': 6.6,
  '2541': 6.8,
  '2551': 8.7,
  '2571': 6.5,
  '2581': 5.45,
  '2588': 6.75,
  '2589': 11.1,
  '2701': 5.95,
  '2721': 13.6,
  '2731': 8.2,
  '2736': 10.45,
  '281': 5.6,
  '284': 9.85,
  '2

### `/predict`

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [60]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 1.85, '102': 9.65, '1203': 3.2, '2259': 13.0, '5073': 3.7}

In [61]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,160.0,70.0
Qty_1203,191.0,138.0
Qty_2259,80.0,38.0
Qty_31,40.0,44.0
Qty_5073,91.0,26.0
